# Randomised Benchmarking

We first generate and queue the circuits we need to execute:

In [ ]:
import lmfit as lm
import matplotlib.pyplot as plt
import numpy as np
from clifford_circuits import clifford_circuits
from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration
from tqdm.auto import tqdm

# Define constants
QUBIT = 4
CLIFFORD_GATES = np.arange(1, 200, 20)
NUM_SEEDS = 20
INVERSE = False

circuits = []

# Run Randomized Benchmarking
for num_cliffords in tqdm(CLIFFORD_GATES, total=len(CLIFFORD_GATES)):
    # We generate the circuits
    circuits.extend(
        clifford_circuits(qubit_idx=QUBIT, num_cliffords=num_cliffords, num_circuits=NUM_SEEDS, inverse=INVERSE)
    )

# We queue all the jobs
result_id = api.execute(circuit=circuits, nshots=1000)

Once the jobs have been executed, we can use the cell below to get the probabilities:

In [ ]:
results = api.get_results(result_id)
results = np.reshape(results, (-1, NUM_SEEDS))  # Reshape results

p0 = []
p0_std = []
p1 = []
p1_std = []

for rb_iteration in results:
    # For each iteration of the RB, we calculate the mean of the probabilities of all the seeds
    temp_p0 = []
    temp_p1 = []

    for result in rb_iteration:
        probs = result["probabilities"]
        temp_p0.append(probs["0"])
        temp_p1.append(probs["1"])

    p0.append(np.mean(temp_p0))
    p0_std.append(np.std(temp_p0))
    p1.append(np.mean(temp_p1))
    p1_std.append(np.std(temp_p1))

And now we can fit the results to get the 1Q-gate fidelity:

In [ ]:
# Define fitting function
def fit(results):
    """Fit the experiment results."""

    NG = 1.875

    def fit_exponential(x, ln_p, A, B):
        return A * np.exp(-ln_p) ** (x) + B

    fit_mod_i = lm.Model(fit_exponential)
    fit_mod_i.set_param_hint("A", value=(results.min()) - (results.max()), vary=True)  # might need .flatten
    fit_mod_i.set_param_hint("B", value=results.max(), vary=True)  # Values were set with the initial_fit_params_I
    fit_mod_i.set_param_hint("ln_p", value=0.1, vary=True)

    pars_i = fit_mod_i.make_params()

    result_i = fit_mod_i.fit(data=results.flatten(), x=CLIFFORD_GATES, params=pars_i)

    ln_p_i = result_i.params["ln_p"].value
    A_I = result_i.params["A"].value
    B_I = result_i.params["B"].value
    # I_fit = fit_exponential(len(rb_i_x), ln_p_I, A_I, B_I)

    xi = np.linspace(CLIFFORD_GATES[0], CLIFFORD_GATES[-1], 1000).tolist()  # To smooth the fit
    i_fit = [fit_exponential(val, ln_p_i, A_I, B_I) for val in xi]

    p_i = np.exp(-ln_p_i)
    avg_clifford_fidelity = (1 + p_i) / 2
    avg_gate_fidelity = avg_clifford_fidelity ** (1 / NG)

    return i_fit, avg_gate_fidelity, avg_clifford_fidelity

In [ ]:
# Fit and plot
I_fit, f_cliff, f_gate = fit(np.array(p0))

plt.errorbar(CLIFFORD_GATES, p0, yerr=p0_std, capsize=10, fmt="o", color="blue", label="Error bars")
plt.plot(CLIFFORD_GATES, p0, "o--", label="Probabilities")
plt.plot(np.linspace(min(CLIFFORD_GATES), max(CLIFFORD_GATES), 1000), I_fit, "r", label=f"Fidelity: {f_gate:.5f}")
plt.xlabel("Number of Clifford gates")
plt.ylabel("P0")
plt.legend()
plt.show()